# 舵机轨迹规划实验
## 导入依赖

In [1]:
# 添加路径
import sys
sys.path.append('../src/')
# 导入依赖
import time
from fs_arm_4dof import Arm4DoF
from config import *
import trapezoid
import s_curve
import minimum_jerk
import logging
import math
import numpy as np

In [28]:
logging.basicConfig(level=logging.WARN)

## 机械臂初始化

In [31]:
arm = Arm4DoF('/dev/ttyUSB1')

In [32]:
arm.get_thetas()

[-0.0, -2.356194490192345, 1.5707963267948966, 0.7853981633974483]

## 测试马上旋转到目标角度

In [44]:
arm.set_joint({0:math.pi/4}, interval=0)

In [56]:
arm.set_joint({0:-math.pi/2}, interval=0)

In [37]:
arm.set_joint({JOINT4:0.60}, interval=0)


## 梯形轨迹规划

In [70]:
def test_trapezoid(joint_id, theta_e):
    theta_s = math.degrees(arm.get_thetas()[joint_id])
    w_max, a_max, t01, t12, t23 = trapezoid.trapezoid_plan1(theta_s, theta_e)
    t_arr, a_arr, w_arr, theta_arr = trapezoid.trapezoid_sequence(theta_s, theta_e, t01, t12, t23, \
                                                    w_max=w_max, a_max=a_max)
    theta_arr_rad = np.radians(theta_arr)
    for theta in theta_arr_rad:
        arm.set_joint({joint_id:theta}, interval=0)
        time.sleep(trapezoid.DELTA_T)
        

In [71]:
test_trapezoid(JOINT1, -90)

In [72]:
test_trapezoid(JOINT1, 90)

## S-Curve

In [66]:
def test_s_curve(joint_id, theta_e):
    theta_s = math.degrees(arm.get_thetas()[joint_id])
    w_s = 0
    w_e = 0
    J_acc = s_curve.REF_MAX_J_ACC
    J_dec = s_curve.REF_MAX_J_DEC
    # 时间规划
    T1,T4,T5 = s_curve.scurve_time_plan(theta_s, theta_e, w_s, w_e, J_acc, J_dec)
    # 求解时间序列
    ret, t_arr, j_arr, a_arr, w_arr, theta_arr = s_curve.scurve_trajectory_seq(theta_s, theta_e, w_s, w_e, J_acc, J_dec, T1, T4, T5)
    theta_arr_rad = np.radians(theta_arr)
    for theta in theta_arr_rad:
        arm.set_joint({joint_id:theta}, interval=0)
        time.sleep(s_curve.DELTA_T)

In [86]:
test_s_curve(JOINT1, 90)

## Minimum Jerk

In [76]:
def test_minimum_jerk(joint_id, theta_e, T):
    theta_s = math.degrees(arm.get_thetas()[joint_id])
    w_s = 0
    w_e = 0
    a_s = 0
    a_e = 0
    # 规划
    c = minimum_jerk.minimum_jerk_plan(theta_s, theta_e, w_s, w_e, a_s, a_e, T)
    t_arr, theta_arr = minimum_jerk.minimum_jerk_seq(T, c)
    # theta_arr = theta_arr.reshape(-1)
    # 执行
    theta_arr_rad = np.radians(theta_arr)
    for theta in theta_arr_rad:
        arm.set_joint({joint_id:theta}, interval=0)
        time.sleep(s_curve.DELTA_T)

In [87]:
test_minimum_jerk(JOINT1, -90, 1.5)